In [4]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
import os
engine = create_engine('postgres://%s:%s@104.154.139.71/%s'%('postgres',os.getenv('db_password'),'tweets_db'))
connection = engine.connect()
import spacy
nlp = spacy.load('en_default')
from numpy import loadtxt
import json
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/Users/ramrajvelmurugan/anaconda/envs/my_projects_env/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


First we import everything ( following this xgboost tutorial: https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/ ). Then lets get the tweets for training and extract features. 


Features that need to be added:

 * Extract OSM node names and then compare uniqueness of OSM nodes at the text level (NoOfDistinctOSMNodes)
 * Match node with entity, find the population and if population field exists for the nodes 
 * Capitalization of the entity names
 


In [9]:
tweets = pd.read_sql_query("""
select 
    tweets.tweet_id,
    tweet_text,
    to_char(tweet_date, 'HH24') as tweet_hour,
    isALocationTweet,
    (
        select count(id) 
        from osmtweetlocalization 
        where osmtweetlocalization.tweet_id= tweets.tweet_id
    ) as NoOfOSMNodes,
    (
        select count(distinct osm_id)
        from osmtweetlocalization
        where osmtweetlocalization.tweet_id = tweets.tweet_id
        
    ) as NoOfDistinctOSMNodes,
    case when lower(users.lang) like 'en%%' then TRUE else FALSE end as langIsEnglish, 
    case when lower(tweet_text) like '%%flood%%' then TRUE else FALSE end as floodkey,
    case when lower(tweet_text) like '%%power%%' then TRUE else FALSE end as powerkey,
    case when lower(tweet_text) like '%%wind%%' then TRUE else FALSE end as windkey,
    case when lower(tweet_text) like '%%water%%' then TRUE else FALSE end as waterkey,
    case when lower(tweet_text) like '%%sewage%%' then TRUE else FALSE end as sewagekey,
    case when lower(tweet_text) like '%%severe%%' then TRUE else FALSE end as severekey,
    case when lower(tweet_text) like '%%tornado%%' then TRUE else FALSE end as tornadokey,
    case when lower(tweet_text) like '%%damage%%' then TRUE else FALSE end as damagekey,
    case when lower(tweet_text) like '%%rain%%' then TRUE else FALSE end as rainkey,
    case when lower(tweet_text) like '%%estate%%' then TRUE else FALSE end as estatekey,
    case when lower(tweet_text) like '%%luxur%%' then TRUE else FALSE end as luxurykey,
    case when lower(tweet_text) like '%%condo%%' then TRUE else FALSE end as condokey,
    case when lower(tweet_text) like '%%sale%%' then TRUE else FALSE end as salekey


from tweetsTrainLocation01  
join tweets on
    tweets.tweet_id = tweetsTrainLocation01.tweet_id
join users on
    tweets.user_id = users.id
where isALocationTweet is not null;
""",engine)
tweets

,tweet_id,tweet_text,tweet_hour,isalocationtweet,noofosmnodes,noofdistinctosmnodes,langisenglish,floodkey,powerkey,windkey,waterkey,sewagekey,severekey,tornadokey,damagekey,rainkey,estatekey,luxurykey,condokey,salekey
0,908203676409454594,"Just posted a photo @ Levy County, Florida htt...",05,True,3,2,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,908034346191392769,Well I'm gonna be home in Aiken till at least ...,18,True,2,1,True,False,False,False,True,False,False,False,False,False,False,False,False,False
2,907002114311905280,How nice is this? I love all the folks at Cre...,22,True,2,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,908293168067760128,I liked a @YouTube video https://t.co/y7DMn5dW...,11,True,4,2,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,908353834086531073,@prageru This is our school district. Okaloosa...,15,True,2,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False
5,908305994047905793,#Residential Price: $255724. Address: 3797 QUA...,12,False,3,2,True,False,False,False,False,False,False,False,False,False,False,False,False,False
6,907813670108803074,Busy night at @SAHSofficial concession stand. ...,03,True,2,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False
7,908216957673111552,Hurricane 360 Spin Mop by BulbHead - Foot-Oper...,06,False,2,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False
8,908203374612541441,Consumer seeking Mobile Home insurance in Ockl...,05,False,1,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False
9,907583877337755649,AudubonFL: RT audubonsociety: Update from Audu...,12,True,1,1,True,False,False,False,False,False,True,False,True,False,False,False,False,False


Now we will add some more spacy-entity based features to this list.

In [19]:

for idx,tweet in tweets.iterrows():
    doc= nlp(tweet.tweet_text)
    tweets.loc[idx,'TotalNoOfEntities'] = len(ent)
    nGPEs =0
    nLOCs=0
    nORGs=0
    nOthers=0
    for ent in doc.ents:
        #print(str(idx),ent.ent_id_,ent.label,ent.label_, ent.text)
        if ent.label == 380:
            nORGs+=1
        elif ent.label== 381:
            nGPEs+=1
        elif ent.label== 382:
            nLOCs+=1
        else:
            nOthers+=1
    tweets.loc[idx,'nGPEentities'] = nGPEs
    tweets.loc[idx,'nLOCentities'] = nLOCs
    tweets.loc[idx,'nORGentities'] = nORGs
    tweets.loc[idx,'nOTHERentities'] = nOthers
    tweets.loc[idx,'nGPEandLOCs'] = nGPEs+nLOCs

        #if ent.label in (380,381,382):
        #     a=2
    



In [62]:
tweets2 = tweets.copy()
#tweets2.drop('tweet_text')
#tweets2.drop('tweet_id')
del tweets2['tweet_text']
del tweets2['tweet_id']

X = tweets2.copy()
del X['isalocationtweet']
X['tweet_hour'] = X['tweet_hour'].astype(int)
Y = tweets2['isalocationtweet']
# split data into train and test sets
seed = 908008
test_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 44.44%


In [64]:
res = y_test.copy()
i= 0 
for idx,result in res.iteritems():
    print('Expected '+str(result)+'; Got '+ str(predictions[i]))
    i+=1

Expected False; Got 0.0
Expected False; Got 0.0
Expected True; Got 1.0
Expected True; Got 0.0
Expected True; Got 0.0
Expected True; Got 0.0
Expected True; Got 0.0
Expected True; Got 1.0
Expected True; Got 0.0
Expected False; Got 1.0
Expected False; Got 0.0
Expected True; Got 0.0
Expected True; Got 1.0
Expected False; Got 1.0
Expected False; Got 0.0
Expected False; Got 0.0
Expected False; Got 0.0
Expected True; Got 0.0
Expected True; Got 1.0
Expected True; Got 0.0
Expected True; Got 0.0
Expected True; Got 1.0
Expected False; Got 1.0
Expected False; Got 1.0
Expected False; Got 1.0
Expected True; Got 1.0
Expected True; Got 0.0


Lets try to categorize the real data now.

In [66]:
tweetstopredict = pd.read_sql_query("""
select 
    tweets.tweet_id,
    tweet_text,
    to_char(tweet_date, 'HH24') as tweet_hour,
    (
        select count(id) 
        from osmtweetlocalization 
        where osmtweetlocalization.tweet_id= tweets.tweet_id
    ) as NoOfOSMNodes,
    (
        select count(distinct osm_id)
        from osmtweetlocalization
        where osmtweetlocalization.tweet_id = tweets.tweet_id
        
    ) as NoOfDistinctOSMNodes,
    case when lower(users.lang) like 'en%%' then TRUE else FALSE end as langIsEnglish, 
    case when lower(tweet_text) like '%%flood%%' then TRUE else FALSE end as floodkey,
    case when lower(tweet_text) like '%%power%%' then TRUE else FALSE end as powerkey,
    case when lower(tweet_text) like '%%wind%%' then TRUE else FALSE end as windkey,
    case when lower(tweet_text) like '%%water%%' then TRUE else FALSE end as waterkey,
    case when lower(tweet_text) like '%%sewage%%' then TRUE else FALSE end as sewagekey,
    case when lower(tweet_text) like '%%severe%%' then TRUE else FALSE end as severekey,
    case when lower(tweet_text) like '%%tornado%%' then TRUE else FALSE end as tornadokey,
    case when lower(tweet_text) like '%%damage%%' then TRUE else FALSE end as damagekey,
    case when lower(tweet_text) like '%%rain%%' then TRUE else FALSE end as rainkey,
    case when lower(tweet_text) like '%%estate%%' then TRUE else FALSE end as estatekey,
    case when lower(tweet_text) like '%%luxur%%' then TRUE else FALSE end as luxurykey,
    case when lower(tweet_text) like '%%condo%%' then TRUE else FALSE end as condokey,
    case when lower(tweet_text) like '%%sale%%' then TRUE else FALSE end as salekey


from tweetsTrainLocation01  
join tweets on
    tweets.tweet_id = tweetsTrainLocation01.tweet_id
join users on
    tweets.user_id = users.id
where isALocationTweet is null;
""",engine)
print('Finished SQL reading.')
for idx,tweet in tweetstopredict.iterrows():
    doc= nlp(tweet.tweet_text)
    tweetstopredict.loc[idx,'TotalNoOfEntities'] = len(ent)
    nGPEs =0
    nLOCs=0
    nORGs=0
    nOthers=0
    for ent in doc.ents:
        #print(str(idx),ent.ent_id_,ent.label,ent.label_, ent.text)
        if ent.label == 380:
            nORGs+=1
        elif ent.label== 381:
            nGPEs+=1
        elif ent.label== 382:
            nLOCs+=1
        else:
            nOthers+=1
    tweetstopredict.loc[idx,'nGPEentities'] = nGPEs
    tweetstopredict.loc[idx,'nLOCentities'] = nLOCs
    tweetstopredict.loc[idx,'nORGentities'] = nORGs
    tweetstopredict.loc[idx,'nOTHERentities'] = nOthers
    tweetstopredict.loc[idx,'nGPEandLOCs'] = nGPEs+nLOCs
print('Finished NER work.')


Finished SQL reading.
Finished NER work.


In [68]:
tweetstopredict2 = tweetstopredict.copy()
del tweetstopredict2['tweet_text']
del tweetstopredict2['tweet_id']
tweetstopredict2['tweet_hour'] = tweetstopredict2['tweet_hour'].astype(int)

y_pred = model.predict(tweetstopredict2)
print('Finished predictions')


Finished predictions


In [82]:
print(len(y_pred))
for idx,tweet in tweetstopredict.iterrows():
    if idx<900:
        continue
    print(idx,y_pred[idx],tweet.tweet_text)
    if idx>1000:
        break

6611
900 True Flagler County:  Coastal Flood Warning issued September 14 at 4:15AM EDT until September 15 at 12:00AM EDT by NWS https://t.co/AjeChos7pB
901 True Hamilton trends now: Irma, Supercrawl, Miss America, Mother, Porsche. https://t.co/DojW9F7x4E
902 False @KurtBusch We here in Hernando County Florida are starting to get power back slowly but surely.. Enjoy watching you race Kurt... GOD BLESS!!
903 False For Florida Keys Residents, Home Was the Ultimate Getaway. Then Irma Hit. https://t.co/GtHsfHKS6Y https://t.co/vdrvGCnI2o
904 False Watching @cnn interviewing victims of Irma in St. Martins. Life is very different, this is Sierra Leone everyday.
905 True Saint Martin (France): Hurricane Irma: Saint Martin - East - Initial Aerial Damage Assessment, 08SEP17… https://t.co/9h6Efy53Ed
906 False Grants to Florida K-12 Teachers for Arts and Culture in Lake, Orange, Osceola, and Seminole Counties-   https://t.co/5KsMncpbdX
907 True Cleared: Flooding in Osceola on I-4 east at Exit 67 SR

Now lets update the table with the predictions.

In [101]:
print("Clearing the prediction column first.")
connection.execute("UPDATE tweetsTrainLocation01 set predictedIsALocationTweet = null;")
for tf in [True,False]:
    sqlUpdateStart = "UPDATE tweetsTrainLocation01 set predictedIsALocationTweet="+str(tf)+" WHERE tweet_id in ("
    #print(sqlUpdateStart)
    updatelist = ""
    for idx,tweet in tweetstopredict.iterrows():
        if y_pred[idx] == tf:
            updatelist = updatelist+str(tweet.tweet_id)+","
        if idx>0 and ((idx%1000)==0 or idx==len(tweetstopredict)-1):
            sqlstring = sqlUpdateStart+updatelist[:-1]+")"
            connection.execute(sqlstring)
            print("Updated 1000 entries.")
            updatelist=""
print('done')            
            


Clearing the prediction column first.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
Updated 1000 entries.
done


Now lets make the GeoJSON file

In [113]:
tweetlocations = pd.read_sql_query("""

  
    select left(tweet_text,140),lat,lon,
   date_part('day',tweet_date) as day,
   date_part('hour',tweet_date) as hour,
   date_part('minute',tweet_date) as minute,   
   case when lower(tweet_text) like '%%flood%%' then 'TRUE' else 'FALSE' end as floodkey,   
   case when lower(tweet_text) like '%%power%%' then 'TRUE' else 'FALSE' end as powerkey,
   case when lower(tweet_text) like '%%rain%%' then 'TRUE' else 'FALSE' end as rainkey,
   tweets.tweet_id::text

from tweets 
join osmtweetlocalization 
    on osmtweetlocalization.tweet_id = tweets.tweet_id 
join planet_osm_nodes
    on osmtweetlocalization.osm_id = planet_osm_nodes.id
join tweetsTrainLocation01
    on tweets.tweet_id = tweetsTrainLocation01.tweet_id
where 
    tweets.tweet_id in (
        select tweet_id 
        from osmtweetlocalization  
        group by tweet_id 
    )
    and (
        predictedIsALocationTweet is True 
        OR
        isALocationTweet is True
    )
    
order by random();



""",engine)
print("Executed the sql.")
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    """
    Thanks to Goeff Boeing
    http://geoffboeing.com/2015/10/exporting-python-data-geojson/
    """
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

tweetlocs2 = tweetlocations.copy()
tweetlocs2.lat = tweetlocs2.lat/10000000
tweetlocs2.lon = tweetlocs2.lon/10000000
tweetlocs2.day = (tweetlocs2.day-8)*24+tweetlocs2.hour
dataout = df_to_geojson(tweetlocs2,['left','hour','day','minute','floodkey','powerkey','rainkey','tweet_id'],lat='lat',lon='lon')
dataout2=dataout
print('Writing geojson file...')
with open('trainedTweetLocations.geojson', 'w') as outfile:
    json.dump(dataout2, outfile,indent=2)
print('done')


Executed the sql.
Writing geojson file...
done


In [112]:
len(tweetlocations)


6153